<a href="https://colab.research.google.com/github/Team7-Sept2021/test/blob/main/ML_Data_Science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler

from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error


import time

In [3]:
df = pd.read_excel("mpg.data.xlsx")
df = df.dropna(axis=1, thresh= 200)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     406 non-null    int64  
 2   displayments  406 non-null    float64
 3   horsepower    400 non-null    float64
 4   weight        406 non-null    int64  
 5   acceleration  406 non-null    float64
 6   model year    406 non-null    int64  
 7   origin        406 non-null    int64  
 8   car name      406 non-null    object 
dtypes: float64(4), int64(4), object(1)
memory usage: 28.7+ KB


# Preprocessing



1.   Handling missing values
2.   Drop the 'car name' feature
3.   Split the Dataset
4.   Encoding the categorical feature 'origin'
5.   Scaling the features




In [4]:
df_re_missing = df.dropna(axis = 0,subset=['mpg'])
df['horsepower'] =df['horsepower'].fillna(df['horsepower'].median())
df_prepro = df_re_missing.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
dict_region = {1:'USA',2:'Europe',3:'Japan'}
df_prepro['origin'] = df_prepro['origin'].replace(dict_region)

In [7]:
y =  df_prepro['mpg']
X = df_prepro.iloc[:,[1,2,3,4,5,6,7]]

In [8]:
seed = 42  


X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=seed)

In [9]:
#OneHotEncoder for X_train
X_train = X_train.reset_index()
encoder = OneHotEncoder( handle_unknown="ignore")
enc = pd.DataFrame(encoder.fit_transform(X_train[['origin']]).toarray())
X_train = X_train.join(enc)
X_train.set_index('index', inplace=True)
#drop 'origin'
X_train = X_train.drop(columns=['origin'])


#OneHotEncoder for X_test
X_test = X_test.reset_index()
enc = pd.DataFrame(encoder.transform(X_test[['origin']]).toarray())
X_test = X_test.join(enc)
X_test.set_index('index', inplace=True)
#drop 'origin'
X_test = X_test.drop(columns=['origin'])

In [11]:
#StandardScaler 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

# ML process

In [12]:
lassoReg = linear_model.Lasso()
linear = LinearRegression()
knr = KNeighborsRegressor()
regr = DecisionTreeRegressor()
svr = SVR()
rfr = RandomForestRegressor()
hgbr = HistGradientBoostingRegressor()
model = [lassoReg, linear, knr, regr, svr, rfr, hgbr]
for i in model:
  i.fit(X_train, y_train)

  pred_test = i.predict(X_test)
  print(i)
  print(f'Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, pred_test))}')
  print(f'Mean Absolute Error: {mean_absolute_error(y_test, pred_test)}')
  print(f'R2:{r2_score(y_test,pred_test)}\n')

  pred_test = i.predict(X_train)
  print(f'Root Mean Squared Error: {np.sqrt(mean_squared_error(y_train, pred_test))}')
  print(f'Mean Absolute Error: {mean_absolute_error(y_train, pred_test)}')
  print(f'R2:{r2_score(y_train, pred_test)}\n\n\n')
  

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)
Root Mean Squared Error: 2.9418163917996827
Mean Absolute Error: 2.2515865750073827
R2:0.8390393137529892

Root Mean Squared Error: 3.6621739121503114
Mean Absolute Error: 2.7094856154159297
R2:0.7860873073894348



LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
Root Mean Squared Error: 2.8876733677169266
Mean Absolute Error: 2.2881587267140917
R2:0.8449096332762084

Root Mean Squared Error: 3.369747091715918
Mean Absolute Error: 2.604740365903637
R2:0.8188854923523801



KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')
Root Mean Squared Error: 2.2763281837204405
Mean Absolute Error: 1.8475000000000001
R2:0.9036263208313828

Ro

# KNeighborsRegressor - Grid

In [13]:
start = time.time()
param_knr_grid = {'n_neighbors': np.arange(3,17, 2),
                  'algorithm':[ 'ball_tree', 'kd_tree', 'brute'],
                  'leaf_size':np.arange(5, 35, 1),
                  'p':[1,2,3]}   


knr = KNeighborsRegressor()
gknr = GridSearchCV(knr, param_knr_grid, cv=KFold(n_splits=100,
                                              shuffle=True, random_state=101))   
gknr.fit(X_train, y_train)

end = time.time()
print(f'{(end - start)/60:.2f}m')

7.91m


In [14]:
pred_test = gknr.predict(X_test)
print(f'Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, pred_test))}')
print(f'Mean Absolute Error: {mean_absolute_error(y_test, pred_test)}')
print(f'R2:{r2_score(y_test,pred_test)}')

Root Mean Squared Error: 2.2463971153827633
Mean Absolute Error: 1.7474999999999998
R2:0.9061440622060856


In [15]:
pred_test = gknr.predict(X_train)
print(f'Root Mean Squared Error: {np.sqrt(mean_squared_error(y_train, pred_test))}')
print(f'Mean Absolute Error: {mean_absolute_error(y_train, pred_test)}')
print(f'R2:{r2_score(y_train, pred_test)}')

Root Mean Squared Error: 2.5227827303203334
Mean Absolute Error: 1.7256603773584904
R2:0.8984877803681359


In [16]:
print(gknr.best_estimator_)

KNeighborsRegressor(algorithm='ball_tree', leaf_size=5, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='uniform')


# HistGradientBoostingRegressor

In [17]:
start = time.time()
param_hgbr_grid = {'learning_rate': np.arange(0.1, 0.6, 0.1),
                  'max_iter': np.arange(90, 110, 5),
                  'min_samples_leaf': np.arange(16, 21, 1)
                  }


hgbr = HistGradientBoostingRegressor()
ghgbr = GridSearchCV(hgbr, param_hgbr_grid, cv=KFold(n_splits=100,
                                              shuffle=True, random_state=101))
ghgbr.fit(X_train, y_train)
end = time.time()
print(f'{(end - start)/60:.2f}m')

29.34m


In [18]:
pred_test = ghgbr.predict(X_test)
print(f'Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, pred_test))}')
print(f'Mean Absolute Error: {mean_absolute_error(y_test, pred_test)}')
print(f'R2:{r2_score(y_test,pred_test)}')

Root Mean Squared Error: 2.433873891004786
Mean Absolute Error: 1.8047991470846205
R2:0.8898245503299671


In [19]:
pred_test = ghgbr.predict(X_train)
print(f'Root Mean Squared Error: {np.sqrt(mean_squared_error(y_train, pred_test))}')
print(f'Mean Absolute Error: {mean_absolute_error(y_train, pred_test)}')
print(f'R2:{r2_score(y_train, pred_test)}')

Root Mean Squared Error: 0.7774757456542041
Mean Absolute Error: 0.5372586039606625
R2:0.990358772644229


In [20]:
print(ghgbr.best_estimator_)

HistGradientBoostingRegressor(l2_regularization=0.0,
                              learning_rate=0.30000000000000004,
                              loss='least_squares', max_bins=255,
                              max_depth=None, max_iter=90, max_leaf_nodes=31,
                              min_samples_leaf=19, n_iter_no_change=None,
                              random_state=None, scoring=None, tol=1e-07,
                              validation_fraction=0.1, verbose=0,
                              warm_start=False)


# RandomForestRegressor - Grid

In [21]:
start = time.time()
param_rfr_grid = {'n_estimators': np.arange(90, 110, 2),
              'max_features' : ['auto', 'sqrt'],
              'min_samples_split' : np.arange(2, 5, 1),
              'min_samples_leaf' : np.arange(1, 4, 1),
              'bootstrap' : [True, False]}


rfr = RandomForestRegressor()
grfr = GridSearchCV(rfr, param_rfr_grid, cv=KFold(n_splits=100,
                                              shuffle=True, random_state=101))   
grfr.fit(X_train, y_train)
end = time.time()
print(f'{(end - start)/60:.2f}m')

98.38m


In [22]:
pred_test = grfr.predict(X_test)
print(f'Mean Absolute Error: {mean_absolute_error(y_test, pred_test)}')
print(f'Score: {grfr.score(X_test, y_test)}')
print(f'R2:{r2_score(y_test,pred_test)}')

Mean Absolute Error: 1.5984227631645573
Score: 0.9172614695075233
R2:0.9172614695075233


In [23]:
pred_test = grfr.predict(X_train)
print(f'Root Mean Squared Error: {np.sqrt(mean_squared_error(y_train, pred_test))}')
print(f'Mean Absolute Error: {mean_absolute_error(y_train, pred_test)}')
print(f'R2:{r2_score(y_train, pred_test)}')

Root Mean Squared Error: 1.048261539932056
Mean Absolute Error: 0.6861156800773511
R2:0.9824733897355683


In [24]:
print(grfr.best_estimator_)

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=2,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=94, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)
